In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud


#for Nlp
import re, nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from nltk import ngrams
stop_words = list(set(stopwords.words('english')))
wordnet_lemmatizer = WordNetLemmatizer()

%matplotlib inline




# First approach
## select random sampels from majority class  (here satsisfied=1) and equal as sample in another class (non satisfied here =0)

In [2]:
reviews_df=pd.read_csv('../input/modeling1-hotel-reviews/data_for_train_model.csv')

In [ ]:
reviews_df.columns

In [3]:
review_non_satisfied=reviews_df[reviews_df["satisfied"]==0]
review_non_satisfied.shape

In [4]:
review_satisfied=reviews_df[reviews_df["satisfied"]==1]
review_satisfied = review_satisfied.sample(n=14636)
review_satisfied.shape

In [5]:
frames = [review_satisfied, review_non_satisfied]
  
all_data = pd.concat(frames)
all_data.shape

In [6]:
import matplotlib.pyplot as plt 
import seaborn as sns
palette = sns.color_palette("bright")
sns.countplot(x='satisfied', data=all_data,palette="Set2")
plt.title("ratio of satisfied");

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data.drop(['review', 'satisfied', 'review_clean'],axis=1) ,all_data["satisfied"], test_size = 0.25, random_state = 42)

In [11]:
rf = RandomForestClassifier(n_estimators = 500,max_depth=None, max_features= "auto", random_state = 42)
rf.fit(X_train, y_train)

In [12]:
y_pred_=rf.predict(X_test)

In [13]:
from sklearn.metrics import confusion_matrix , classification_report
from mlxtend.plotting import plot_confusion_matrix
conf = confusion_matrix(y_test , y_pred_)
plot_confusion_matrix(conf)

# plot_confusion_matrix(conf)
print (classification_report(y_test , y_pred_))

In [ ]:
sns.heatmap(conf, square=True, annot=True, cmap='PuBuGn', fmt='d', cbar=False)

# second approach
## train classifiers on positive and label positive (=1) and negative Reviews with label negative (=0)

In [14]:
df=pd.read_csv('../input/data-preprocessing-eda-hotel-reviews/data_hotel_reviews_clean.csv')

In [ ]:
df.columns

In [15]:
#loading the positive reviews and negative reviews to a single column as text
pos_reviews = df['Positive_Review_clean'].values
pos_reviews = pos_reviews.tolist()
neg_reviews = df['Negative_Review_clean'].values
neg_reviews = neg_reviews.tolist()
text = pos_reviews+neg_reviews

In [16]:
#providing score attribute to the review
score = ['positive' for i in range(len(pos_reviews))]
score += ['negative' for i in range(len(neg_reviews))]
#performing one-hot encoding to the score attrubute.(1- positive and 0- negative)
for i in range(0,len(score)):
    if score[i] == 'positive':
        score[i] = 1
    else:
        score[i] = 0

In [17]:
#loading required data to dataframe.
text_df = pd.DataFrame()
text_df['reviews_clean'] = text
text_df['score'] = score
text_df.head()

In [ ]:
# from nltk.corpus import wordnet

# import string
# from nltk import pos_tag
# from nltk.corpus import stopwords
# from nltk.tokenize import WhitespaceTokenizer
# from nltk.stem import WordNetLemmatizer

# def clean_text(text):
#     # lower text
#     text = text.lower()
#     # tokenize text and remove puncutation
#     text = [word.strip(string.punctuation) for word in text.split(" ")]
#     # remove words that contain numbers
#     text = [word for word in text if not any(c.isdigit() for c in word)]
#     # remove stop words
#     stop = stopwords.words('english')
#     text = [x for x in text if x not in stop]
#     # remove empty tokens
#     text = [t for t in text if len(t) > 0]
#     # pos tag text
#     text =[wordnet_lemmatizer.lemmatize(x) for x in text]
#     # remove words with only one letter
#     text = [t for t in text if len(t) > 1]
#     # join all
#     text = " ".join(text)
#     return(text)

# # clean text data
# text_df["reviews_clean"] = text_df["reviews"].apply(lambda x: clean_text(x))


In [ ]:
text_df.columns

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import time

In [20]:
text_df.dropna(inplace=True)

In [ ]:
score = text_df['score'].values
text_clean=text_df["reviews_clean"].values

In [22]:
# Hyperparameter tuning for MultinomialNB with Bigrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
start_time = time.time()
best_params = []
parameters = {'alpha':[i for i in range(1,100,10)]}
acc = []
score = list(score)
for i in range(2000,14000,1000):
    vec = TfidfVectorizer(max_features = i)
    data = vec.fit_transform(text_clean)
    nb = MultinomialNB()
    clf = GridSearchCV(nb, parameters,cv=5)
    x_train, x_test, y_train, y_test = train_test_split(data, score, test_size=0.3, random_state=42)
    clf.fit(x_train, y_train)
    acc.append(100.0*sum(clf.predict(x_test))/len((clf.predict(x_test))))
    best_params.append(clf.best_params_)

In [23]:
start_time = time.time()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix , classification_report


score_Log_reg = []
y_pred = clf.predict(x_test)
conf_NB = confusion_matrix(y_test, y_pred)    

print("Confusion matrix:\n",conf_NB)

#ROC for a given alpha for NB
from sklearn.metrics import roc_curve, auc
# Compute ROC curve and ROC area for each class
probs = clf.predict_proba(x_test)
preds = probs[:,1]
fpr, tpr, threshold = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

#Plot ROC
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#print the log loss
a = log_loss(y_test, probs)
print("The log loss for the Naive bayes is:",a)


#Precision and recall
tn = conf_NB[0,0]; fp = conf_NB[0,1]; fn = conf_NB[1,0]; tp = conf_NB[1,1];

precision = 100*float(tp)/(tp+fp)
recall = 100*float(tp)/(tp+fn)

print("Precision :",precision)
print("Recall :",recall)
print("classification_report",classification_report(y_test , y_pred))

tp = conf_NB[0][0]
tn = conf_NB[1][1]
print("The accuracy is {} %".format(round(100.0*(tp+tn)/len(y_test),2)))
print('------------ %s seconds ------------'%(time.time()-start_time))


## if we want to try more classifiers 

In [ ]:
tfidf = TfidfVectorizer(max_features=10000)
tfidf_result = tfidf.fit_transform(text_df["reviews_clean"]).toarray()

In [ ]:
tfidf_result.shape,len(score)

In [ ]:
    x_train, x_test, y_train, y_test = train_test_split(tfidf_result, score, test_size=0.3, random_state=42)


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid={
 'max_depth': [None],
 'max_features': ['auto'],
 'n_estimators': [100, 200, 300, 400]}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

grid = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1,verbose=1)


In [ ]:
grid_result = grid.fit(x_train, y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
model=grid_result.best_estimator_
model_pre=model.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
conf = confusion_matrix(y_test , model_pre)
from mlxtend.plotting import plot_confusion_matrix
plot_confusion_matrix(conf)
print (classification_report(y_test , model_pre))